In [6]:
import pandas as pd
import requests
import networkx as nx
import random
from collections import Counter
import plotly.graph_objects as go

In [7]:
url = "https://crowding.data.tfl.gov.uk/NUMBAT/NUMBAT%202023/NBT23FRI_outputs.xlsx"

r = requests.get(url)
with open("NBT23FRI_outputs.xlsx", "wb") as f:
    f.write(r.content)

xls = pd.ExcelFile("NBT23FRI_outputs.xlsx")
print(xls.sheet_names)

['_Cover', 'Link_Loads', 'Link_Frequencies', 'Line_Boarders', 'Station_Flows', 'Station_Entries', 'Station_Exits', 'Station_Boarders', 'Station_Alighters']


In [14]:
df_links = pd.read_excel(xls, sheet_name="Link_Loads", header=2)
df_flows = pd.read_excel(xls, sheet_name="Station_Flows", header=2)
df_entries = pd.read_excel(xls, sheet_name="Station_Entries", header=2)
df_exits = pd.read_excel(xls, sheet_name="Station_Exits", header=2)
df_boarders = pd.read_excel(xls, sheet_name="Station_Boarders", header=2)
df_alighters = pd.read_excel(xls, sheet_name="Station_Alighters", header=2)

def extract_interchange_stations(df_flows):
    interchange_rows = df_flows[df_flows["Movement"].str.contains("Interchange", na=False)]
    interchange_stations = sorted(interchange_rows["To Station"].dropna().unique())
    return interchange_stations

def build_graph_from_links(df_links, entry_stations=None, exit_stations=None):
    G = nx.DiGraph()
    for _, r in df_links.iterrows():
        G.add_edge(r["From Station"], r["To Station"], weight=r["Total"])
    if len(entry_stations) > 0:
        for station in entry_stations:
            if station not in G:
                G.add_node(station)
    if len(exit_stations) > 0:
        for station in exit_stations:
            if station not in G:
                G.add_node(station)
    return G

def build_entry_weights(df_entries):
    entries = df_entries[["Station", "Total"]].dropna()
    return dict(zip(entries["Station"], entries["Total"]))

def build_exit_weights(df_exits):
    exits = df_exits[["Station", "Total"]].dropna()
    return dict(zip(exits["Station"], exits["Total"]))

def build_board_alight_dicts(df_boarders, df_alighters):
    board_dict = df_boarders.groupby("Station")["Total"].sum().to_dict()
    alight_dict = df_alighters.groupby("Station")["Total"].sum().to_dict()
    return board_dict, alight_dict

def simulate_route(G, start, interchange_list, board_dict, alight_dict,
                   max_steps=5, transfer_prob=0.3):
    path = [start]
    cur = start
    score = 0
    for _ in range(max_steps):
        if cur in interchange_list and random.random() < transfer_prob:
            candidates = [s for s in G.successors(cur) if s in interchange_list and s not in path]
            if candidates:
                nxt = random.choice(candidates)
                path.append(f"[TRANSFER→]{nxt}")
                cur = nxt
                continue

        if cur not in G:
            break
        succ = [s for s in G.successors(cur) if s not in path]
        if not succ:
            break
        weights = [G[cur][s]["weight"] for s in succ]
        cur = random.choices(succ, weights=weights, k=1)[0]
        path.append(cur)
        score += board_dict.get(cur, 0) + alight_dict.get(cur, 0)
    return path, score

def run_simulation_pipeline(df_links, df_flows, df_entries, df_exits, df_boarders, df_alighters,
                            num_routes=10000):
    interchange_stations = extract_interchange_stations(df_flows)
    entry_stations = df_entries["Station"].dropna().unique()
    exit_stations = df_exits["Station"].dropna().unique()
    G = build_graph_from_links(df_links, entry_stations, exit_stations)
    entry_weights = build_entry_weights(df_entries)
    exit_weights = build_exit_weights(df_exits)
    board_dict, alight_dict = build_board_alight_dicts(df_boarders, df_alighters)

    starts = list(entry_weights.keys())
    weights = list(entry_weights.values())

    routes = []
    scores = []
    ends = []

    for _ in range(num_routes):
        start = random.choices(starts, weights=weights, k=1)[0]
        path, score = simulate_route(G, start, interchange_stations, board_dict, alight_dict)
        routes.append(path)
        scores.append(score)
        clean_path = [step for step in path if not str(step).startswith("[TRANSFER")]
        end = clean_path[-1] if len(clean_path) >= 2 else None
        ends.append(end)

    df_result = pd.DataFrame({
        "Route": routes,
        "Score": scores,
        "End": ends
    })

    df_result["End Exit Volume"] = df_result["End"].map(exit_weights)
    return df_result

def analyze_results(df_result):
    # Top scored routes
    top_routes = df_result.sort_values(by="Score", ascending=False).head(20)

    all_links = []

    for route in df_result["Route"].tolist():
        clean_steps = []
        for step in route:
            if isinstance(step, str) and step.startswith("[TRANSFER→]"):
                clean_steps.append(step.replace("[TRANSFER→]", ""))
            else:
                clean_steps.append(step)
        all_links += [(clean_steps[i], clean_steps[i+1]) for i in range(len(clean_steps)-1) if clean_steps[i] != clean_steps[i + 1]]

    pair_counts = Counter(all_links)
    df_pairs = pd.DataFrame(pair_counts.items(), columns=["Step-Step", "Count"]).sort_values(by="Count", ascending=False)

    top_links = df_pairs.head(20)
    sources = [pair[0] for pair in top_links["Step-Step"]]
    targets = [pair[1] for pair in top_links["Step-Step"]]
    values = top_links["Count"].tolist()

    label_list = list(set(sources + targets))
    label_map = {label: idx for idx, label in enumerate(label_list)}
    source_idx = [label_map[s] for s in sources]
    target_idx = [label_map[t] for t in targets]

    sankey_fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=label_list,
        ),
        link=dict(
            source=source_idx,
            target=target_idx,
            value=values,
        ))])
    sankey_fig.update_layout(title_text="Top 20 Simulated Start-End Routes", font_size=10)

    print("\nTop 20 Scored Routes:")
    print(top_routes[["Route", "Score"]].reset_index(drop=True))

    print("\nTop 20 Start-End Pairs:")
    print(df_pairs.head(20).reset_index(drop=True))

    return sankey_fig

In [15]:
df_result = run_simulation_pipeline(
    df_links, df_flows, df_entries, df_exits,
    df_boarders, df_alighters,
    num_routes=10000
)
display(df_result.head())

,Route,Score,End,End Exit Volume
0,"[Stratford, [TRANSFER→]Mile End, Bethnal Green...",499626.611665,Shoreditch High Street,12921.528778
1,"[Old Street, Angel, King's Cross St. Pancras, ...",818890.530022,Farringdon,64227.322636
2,"[Canary Wharf LU, Canada Water, Bermondsey, Lo...",611095.625162,Liverpool Street LU,74058.988265
3,"[Mornington Crescent, Camden Town (Edgware), E...",811378.172293,Holborn,34501.549346
4,"[Ilford, Seven Kings, Goodmayes, Chadwell Heat...",82043.877558,Emerson Park,385.007400


In [16]:
fig = analyze_results(df_result)
fig.show()


Top 20 Scored Routes:
                                                Route         Score
0   [Pimlico, Victoria LU, Green Park, Oxford Circ...  1.553579e+06
1   [Highbury & Islington, King's Cross St. Pancra...  1.483416e+06
2   [Hyde Park Corner, Green Park, Oxford Circus, ...  1.396390e+06
3   [Baker Street, Bond Street, Oxford Circus, Tot...  1.391344e+06
4   [St. James's Park, Victoria LU, Green Park, Ox...  1.386418e+06
5   [Victoria LU, Green Park, Oxford Circus, Warre...  1.360803e+06
6   [Russell Square, King's Cross St. Pancras, Eus...  1.360803e+06
7   [Victoria LU, Green Park, Piccadilly Circus, O...  1.359707e+06
8   [Tottenham Court Road, Bond Street, Oxford Cir...  1.355757e+06
9   [Vauxhall LU, Pimlico, Victoria LU, Green Park...  1.345944e+06
10  [St. James's Park, Victoria LU, Green Park, Ox...  1.339808e+06
11  [Piccadilly Circus, Oxford Circus, Tottenham C...  1.325445e+06
12  [Angel, King's Cross St. Pancras, Russell Squa...  1.325445e+06
13  [King's Cross St. Pan